# Training RL om Cartpole in Balans te Houden

Dit notebook maakt deel uit van het [AI for Beginners Curriculum](http://aka.ms/ai-beginners). Het is geïnspireerd door de [officiële PyTorch-tutorial](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) en [deze Cartpole PyTorch-implementatie](https://github.com/yc930401/Actor-Critic-pytorch).

In dit voorbeeld gebruiken we RL om een model te trainen dat een paal in balans houdt op een karretje dat naar links en rechts kan bewegen op een horizontale schaal. We maken gebruik van de [OpenAI Gym](https://www.gymlibrary.ml/) omgeving om de paal te simuleren.

> **Opmerking**: Je kunt de code van deze les lokaal uitvoeren (bijvoorbeeld vanuit Visual Studio Code), in welk geval de simulatie in een nieuw venster wordt geopend. Als je de code online uitvoert, moet je mogelijk enkele aanpassingen aan de code doen, zoals beschreven [hier](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

We beginnen met ervoor te zorgen dat Gym is geïnstalleerd:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Laten we de CartPole-omgeving creëren en bekijken hoe we ermee kunnen werken. Een omgeving heeft de volgende eigenschappen:

* **Actieruimte** is de verzameling van mogelijke acties die we bij elke stap van de simulatie kunnen uitvoeren  
* **Observatieruimte** is de ruimte van waarnemingen die we kunnen doen  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Laten we kijken hoe de simulatie werkt. De volgende lus voert de simulatie uit totdat `env.step` de beëindigingsvlag `done` retourneert. We zullen willekeurige acties kiezen met behulp van `env.action_space.sample()`, wat betekent dat het experiment waarschijnlijk heel snel zal mislukken (de CartPole-omgeving stopt wanneer de snelheid van de CartPole, zijn positie of hoek buiten bepaalde grenzen vallen).

> De simulatie wordt geopend in een nieuw venster. Je kunt de code meerdere keren uitvoeren en zien hoe deze zich gedraagt.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Je kunt zien dat de observaties bestaan uit 4 getallen. Deze zijn:
- Positie van het karretje
- Snelheid van het karretje
- Hoek van de paal
- Rotatiesnelheid van de paal

`rew` is de beloning die we bij elke stap ontvangen. In de CartPole-omgeving krijg je 1 punt beloning voor elke simulatiestap, en het doel is om de totale beloning te maximaliseren, oftewel de tijd dat de CartPole in balans blijft zonder om te vallen.

Tijdens reinforcement learning is ons doel om een **beleid** $\pi$ te trainen, dat ons voor elke toestand $s$ vertelt welke actie $a$ we moeten nemen, dus in essentie $a = \pi(s)$.

Als je een probabilistische oplossing wilt, kun je het beleid zien als het teruggeven van een set kansen voor elke actie, oftewel $\pi(a|s)$ zou betekenen de kans dat we actie $a$ moeten nemen in toestand $s$.

## Policy Gradient Methode

In het eenvoudigste RL-algoritme, genaamd **Policy Gradient**, trainen we een neuraal netwerk om de volgende actie te voorspellen.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

We zullen het netwerk trainen door veel experimenten uit te voeren en ons netwerk na elke run bij te werken. Laten we een functie definiëren die het experiment uitvoert en de resultaten retourneert (de zogenaamde **trace**) - alle toestanden, acties (en hun aanbevolen waarschijnlijkheden) en beloningen:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Je kunt één episode uitvoeren met een niet-getraind netwerk en observeren dat de totale beloning (oftewel de lengte van de episode) erg laag is:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Een van de lastige aspecten van het policy gradient-algoritme is het gebruik van **gedisconteerde beloningen**. Het idee is dat we de vector van totale beloningen berekenen bij elke stap van het spel, en tijdens dit proces disconteren we de vroege beloningen met een bepaalde coëfficiënt $gamma$. We normaliseren ook de resulterende vector, omdat we deze zullen gebruiken als gewicht om onze training te beïnvloeden:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Nu gaan we echt trainen! We zullen 300 episodes uitvoeren, en bij elke episode doen we het volgende:

1. Voer het experiment uit en verzamel de trace.
2. Bereken het verschil (`gradients`) tussen de genomen acties en de voorspelde waarschijnlijkheden. Hoe kleiner het verschil, hoe zekerder we zijn dat we de juiste actie hebben genomen.
3. Bereken de verdisconteerde beloningen en vermenigvuldig de gradients met de verdisconteerde beloningen - dit zorgt ervoor dat stappen met hogere beloningen meer invloed hebben op het eindresultaat dan stappen met lagere beloningen.
4. De verwachte doelacties voor ons neuraal netwerk worden deels gehaald uit de voorspelde waarschijnlijkheden tijdens de uitvoering, en deels uit de berekende gradients. We gebruiken de parameter `alpha` om te bepalen in welke mate gradients en beloningen worden meegewogen - dit wordt de *leersnelheid* van het versterkingsalgoritme genoemd.
5. Tot slot trainen we ons netwerk op basis van de toestanden en verwachte acties, en herhalen we het proces.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Laten we nu de aflevering uitvoeren met rendering om het resultaat te zien:


In [ ]:
_ = run_episode(render=True)

Hopelijk kun je zien dat de paal nu behoorlijk goed in balans kan blijven!

## Actor-Critic Model

Het Actor-Critic model is een verdere ontwikkeling van policy gradients, waarbij we een neuraal netwerk bouwen om zowel het beleid als de geschatte beloningen te leren. Het netwerk zal twee outputs hebben (of je kunt het zien als twee aparte netwerken):
* **Actor** zal de actie aanbevelen die moet worden ondernomen door ons de waarschijnlijkheidsverdeling van de toestand te geven, zoals in het policy gradient model.
* **Critic** zou inschatten wat de beloning zou zijn van die acties. Het geeft de totale geschatte beloningen in de toekomst terug bij de gegeven toestand.

Laten we zo'n model definiëren:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

We zouden onze functies `discounted_rewards` en `run_episode` enigszins moeten aanpassen:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Nu gaan we de hoofdtrainingslus uitvoeren. We zullen het handmatige netwerktrainingsproces gebruiken door de juiste verliesfuncties te berekenen en de netwerkparameters bij te werken:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Belangrijkste punten

We hebben in deze demo twee RL-algoritmen gezien: eenvoudige policy gradient en het meer geavanceerde actor-critic. Je kunt zien dat deze algoritmen werken met abstracte begrippen zoals toestand, actie en beloning - hierdoor kunnen ze worden toegepast op zeer verschillende omgevingen.

Reinforcement learning stelt ons in staat om de beste strategie te leren om een probleem op te lossen, enkel door naar de uiteindelijke beloning te kijken. Het feit dat we geen gelabelde datasets nodig hebben, stelt ons in staat om simulaties meerdere keren te herhalen om onze modellen te optimaliseren. Er zijn echter nog steeds veel uitdagingen in RL, die je kunt ontdekken als je besluit je meer te verdiepen in dit interessante gebied van AI.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we ons best doen voor nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
